## Credit Approval Application.

Banks give loans to applicants.....


Plans and process
- Load dataset and view 
- analyse the column types and values (numerical or non numberical)
- Preprocess the dataset to ensure the ML model we choose can make good predictions
- Do some data analysis to build our instuition
- Build a machine learning model that cna predict if an individuals Credit Application will be approved or not 

In [2]:
import pandas as pd
import numpy as np
import matplotlib as plt 



In [3]:
cc_apps = pd.read_csv("D:/A work/DS/Credit_card_approval/Data-Science-Credit-Card-Approvals-Quant/datasets/cc_approvals.data", header= None)

### 1. Inspecting the data

In [4]:
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [5]:
cc_apps.columns

Int64Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], dtype='int64')

The data has been anonymized to protect the privacy but this post here (http://rstudio-pubs-static.s3.amazonaws.com/73039_9946de135c0a49daa7a0a9eda4a67a72.html) explains the data. From the post we se that we have the following columns:
- Gender, Age, Debt, Married, BankCustomer, EducationLevel, Ethnicity, YearsEmployed, PriorDefault, Employed, CreditScore, DriversLicense, Citizen, ZipCode, Income and finally the ApprovalStatus



In [6]:
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print("\n")

cc_apps_info = cc_apps.info()
print(cc_apps_info)


               2           7          10             14
count  690.000000  690.000000  690.00000     690.000000
mean     4.758725    2.223406    2.40000    1017.385507
std      4.978163    3.346513    4.86294    5210.102598
min      0.000000    0.000000    0.00000       0.000000
25%      1.000000    0.165000    0.00000       0.000000
50%      2.750000    1.000000    0.00000       5.000000
75%      7.207500    2.625000    3.00000     395.500000
max     28.000000   28.500000   67.00000  100000.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
0     690 non-null object
1     690 non-null object
2     690 non-null float64
3     690 non-null object
4     690 non-null object
5     690 non-null object
6     690 non-null object
7     690 non-null float64
8     690 non-null object
9     690 non-null object
10    690 non-null int64
11    690 non-null object
12    690 non-null object
13    690 non-null object
14    690 non-null int64

In [7]:
# Renaming the columns to make the process readable 
cc_apps_copy = cc_apps.copy()

cc_apps_copy.columns = [
    'gender','age','debt','married','bankCustomer','educationalLevel','ethinicy','yearsEmployed','priorDefault','employed','creditScore','driversLicense','citizen','zipCode','income','approved']

cc_apps_copy.head()

,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
0,b,30.83,0.000,u,g,w,v,1.25,t,t,1,f,g,00202,0,+
1,a,58.67,4.460,u,g,q,h,3.04,t,t,6,f,g,00043,560,+
2,a,24.50,0.500,u,g,q,h,1.50,t,f,0,f,g,00280,824,+
3,b,27.83,1.540,u,g,w,v,3.75,t,t,5,t,g,00100,3,+
4,b,20.17,5.625,u,g,w,v,1.71,t,f,0,f,s,00120,0,+


In [8]:
cc_apps_copy['bankCustomer'].unique()

array(['g', 'p', '?', 'gg'], dtype=object)

In [9]:
cc_apps[4].unique()

array(['g', 'p', '?', 'gg'], dtype=object)

### 2. Handling missing values

- The dataset contains both numeric and non-numeric data (specifically data that are of float64, int64 and object types). Specifically, the features debt, yearsEmployed, creditScore and income contain numeric values (of types float64, float64, int64 and int64 respectively) and all the other features contain non-numeric values.
- The dataset also contains values from several ranges. Some features have a value range of 0 - 28, some have a range of 2 - 67, and some have a range of 1017 - 100000. Apart from these, we can get useful statistical information (like mean, max, and min) about the features that have numerical values.
- Finally, the dataset has missing values. The missing values in the dataset are labeled with '?'



In [10]:
cc_apps_copy.tail(10)

,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
680,b,19.50,0.290,u,g,k,v,0.290,f,f,0,f,g,00280,364,-
681,b,27.83,1.000,y,p,d,h,3.000,f,f,0,f,g,00176,537,-
682,b,17.08,3.290,u,g,i,v,0.335,f,f,0,t,g,00140,2,-
683,b,36.42,0.750,y,p,d,v,0.585,f,f,0,f,g,00240,3,-
684,b,40.58,3.290,u,g,m,v,3.500,f,f,0,t,s,00400,0,-
685,b,21.08,10.085,y,p,e,h,1.250,f,f,0,f,g,00260,0,-
686,a,22.67,0.750,u,g,c,v,2.000,f,t,2,t,g,00200,394,-
687,a,25.25,13.500,y,p,ff,ff,2.000,f,t,1,t,g,00200,1,-
688,b,17.92,0.205,u,g,aa,v,0.040,f,f,0,f,g,00280,750,-
689,b,35.00,3.375,u,g,c,h,8.290,f,f,0,t,g,00000,0,-


In [11]:
import numpy as np


print(cc_apps_copy.isnull().values.sum())

# replace ? with NaN
cc_apps_copy = cc_apps_copy.replace('?', np.nan)

#inspect the missing values again
print('Toatal Nan: ' + str(cc_apps_copy.isnull().values.sum()))
print('NaN by column:' '\n')  
print(cc_apps.isnull().sum())
cc_apps_copy.tail(20)

0
Toatal Nan: 67
NaN by column:

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64


,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,driversLicense,citizen,zipCode,income,approved
670,b,47.17,5.835,u,g,w,v,5.500,f,f,0,f,g,00465,150,-
671,b,25.83,12.835,u,g,cc,v,0.500,f,f,0,f,g,00000,2,-
672,a,50.25,0.835,u,g,aa,v,0.500,f,f,0,t,g,00240,117,-
673,NaN,29.50,2.000,y,p,e,h,2.000,f,f,0,f,g,00256,17,-
674,a,37.33,2.500,u,g,i,h,0.210,f,f,0,f,g,00260,246,-
675,a,41.58,1.040,u,g,aa,v,0.665,f,f,0,f,g,00240,237,-
676,a,30.58,10.665,u,g,q,h,0.085,f,t,12,t,g,00129,3,-
677,b,19.42,7.250,u,g,m,v,0.040,f,t,1,f,g,00100,1,-
678,a,17.92,10.210,u,g,ff,ff,0.000,f,f,0,f,g,00000,50,-
679,a,20.08,1.250,u,g,c,v,0.000,f,f,0,f,g,00000,0,-


#### Handling missing values for numeric columns.

It is important to handle missing values before we apply any models. Dropping them may lead to loss of information and we don't want that. One common way to handle missing values is to input the mean values - mean imputation

In [12]:
cc_apps_copy.fillna(cc_apps_copy.mean(), inplace=True)

# Count the number of NaNs in the dataset to verify
print('Total NaN: ' + str(cc_apps.isnull().values.sum()))
cc_apps.isnull().sum()

Total NaN: 0


0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
dtype: int64

#### Handling missing values for non-numeric columns.

We are going to impute these missing values with the most frequent values as present in the respective columns. This is good practice when it comes to imputing missing values for categorical data in general.

In [13]:
for col in cc_apps_copy:
    # check if the column type is 'object
    if cc_apps_copy[col].dtypes == 'object':
        # impute the mode value
        cc_apps_copy = cc_apps_copy.fillna(cc_apps_copy[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print('Total missing values:' + str(cc_apps_copy.isnull().values.sum()))
print('Missing values in each column:')
cc_apps_copy.isnull().sum()

Total missing values:0
Missing values in each column:


gender              0
age                 0
debt                0
married             0
bankCustomer        0
educationalLevel    0
ethinicy            0
yearsEmployed       0
priorDefault        0
employed            0
creditScore         0
driversLicense      0
citizen             0
zipCode             0
income              0
approved            0
dtype: int64

#### 3. Preprocessing the data 
Before we proceed to the ml model, we will need to do the following;
-Convert the non-numeric data into numeric.
-Split the data into train and test sets.
-Scale the feature values to a uniform range.



In [14]:
from sklearn.preprocessing import LabelEncoder

# initialize LabelEncoder
label = LabelEncoder()

for col in cc_apps_copy:
    if cc_apps_copy[col].dtypes == 'object':
            # Use LabelEncoder to do the numeric transformation
        label.fit(cc_apps_copy[col])
        cc_apps_copy[col] = label.transform(cc_apps_copy[col])

cc_apps_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 16 columns):
gender              690 non-null int32
age                 690 non-null int32
debt                690 non-null float64
married             690 non-null int32
bankCustomer        690 non-null int32
educationalLevel    690 non-null int32
ethinicy            690 non-null int32
yearsEmployed       690 non-null float64
priorDefault        690 non-null int32
employed            690 non-null int32
creditScore         690 non-null int64
driversLicense      690 non-null int32
citizen             690 non-null int32
zipCode             690 non-null int32
income              690 non-null int64
approved            690 non-null int32
dtypes: float64(2), int32(12), int64(2)
memory usage: 54.0 KB


### 4. Preprocessing 2 - Spliting the dataset into train and test datasets
In ML, it is best to split the dataset into two, this helps us used different datasets to train and test the model. The model should be tested on a completely different dataset from the one used to train the model.

Before we split the dataset, we ought to do some feature selection. In this case we will simply drop the "driverLicense" and "zipCode" columns as they don't affect the credit card decision.

In [16]:
# import train split
from sklearn.model_selection import train_test_split

# drop unwanted columns
cc_apps_copy = cc_apps_copy.drop(['driversLicense', 'zipCode'], axis=1)
cc_apps_copy.head(5)

,gender,age,debt,married,bankCustomer,educationalLevel,ethinicy,yearsEmployed,priorDefault,employed,creditScore,citizen,income,approved
0,1,156,0.000,2,1,13,8,1.25,1,1,1,0,0,0
1,0,328,4.460,2,1,11,4,3.04,1,1,6,0,560,0
2,0,89,0.500,2,1,11,4,1.50,1,0,0,0,824,0
3,1,125,1.540,2,1,13,8,3.75,1,1,5,0,3,0
4,1,43,5.625,2,1,13,8,1.71,1,0,0,2,0,0


In [21]:
# convert into a numpy array
cc_apps_np = cc_apps_copy.values

# Segregating features and the labels into seperate variables
X, y = cc_apps_np[:, 0:12], cc_apps_np[:, 13]

In [23]:
# split into train and test set

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

### 5. Preprocessing - Scaling the dataset
The data is now split into two separate sets - train and test sets respectively. Thefinal preprocessing step of scaling must be done before we can fit a machine learning model to the data.

What does scaled data mean? Let's use CreditScore as an example. The credit score of a person is their creditworthiness based on their credit history. The higher this number, the more financially trustworthy a person is considered to be. So, a CreditScore of 1 is the highest since we're rescaling all the values to the range of 0-1.

In [24]:
# import minmaxscaler
from sklearn.preprocessing import MinMaxScaler

# Instantiate MinMaxScaler and use that to scale X_train and X_test
scaler = MinMaxScaler(feature_range=(0,1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.fit_transform(X_test)

Predicting if a credit card application will be approved or not is a classification task. 

According to UCI, the dataset contains more instances that correspond to "Denied" status than instances corresponding to "Approved" status. Specifically, out of 690 instances, there are 383 (55.5%) applications that got denied and 307 (44.5%) applications that got approved.

This gives us a benchmark. A good machine learning model should be able to accurately predict the status of the applications with respect to these statistics.

Starting our machine learning modeling with a Logistic Regression model (a generalized linear model)

In [25]:
# import logistic regre
from sklearn.linear_model import LogisticRegression

# instantiate a LogisticRegression classifier with default params 
logreg = LogisticRegression()

#fit logreg to the dataset
logreg.fit(rescaledX_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

### 6. Making predictions and evaluating performance

But how well does the model perform?

We will now evaluate our model on the test set with respect to classification accuracy. But we will also take a look the model's confusion matrix. 

In the case of predicting credit card applications, it is equally important to see if our machine learning model is able to predict the approval status of the applications as denied that originally got denied. If our model is not performing well in this aspect, then it might end up approving the application that shouldn't have been approved. The confusion matrix helps us to view our model's performance from these aspects. 

In [26]:
# import confusion matrix

from sklearn.metrics import confusion_matrix
# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
logreg_acc = logreg.score(rescaledX_test, y_test)

print("Accuracy of logistic regression classifier: ", logreg_acc)

Accuracy of logistic regression classifier:  0.8377192982456141


In [27]:
# Print the confusion matrix of the logreg model
logreg_conf = confusion_matrix(y_test, y_pred)
print('Confusion matrix: \n ', logreg_conf)


Confusion matrix: 
  [[92 11]
 [26 99]]


The model was able to yield an accuracy score of almost 84%.

For the confusion matrix, the first element of the of the first row of the confusion matrix denotes the true negatives meaning the number of negative instances (denied applications) predicted by the model correctly. And the last element of the second row of the confusion matrix denotes the true positives meaning the number of positive instances (approved applications) predicted by the model correctly.

Let's see if we can do better. We can perform a grid search of the model parameters to improve the model's ability to predict credit card approvals.

scikit-learn's implementation of logistic regression consists of different hyperparameters but we will grid search over the following two:

    tol
    max_iter


In [28]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV
# Define the grid of values for tol and max_iter
tol = [0.01, 0.001, 0.0001]
max_iter = [100, 150, 200]

# Create a dictionary where tol and max_iter are keys and the lists of their values are corresponding values
param_grid = dict(tol= tol, max_iter= max_iter)

param_grid

{'tol': [0.01, 0.001, 0.0001], 'max_iter': [100, 150, 200]}

### 7. Finding the best performing model

The grid of hyperparameter values has been defined and converted into a single dictionary format which GridSearchCV() expects as one of its parameters. Now, we will begin the grid search to see which values perform best.

We will instantiate GridSearchCV() with our earlier logreg model with all the data we have. Instead of passing train and test sets separately, we will supply X (scaled version) and y. We will also instruct GridSearchCV() to perform a cross-validation of five folds.

We'll end the notebook by storing the best-achieved score and the respective best parameters.

While building this credit card predictor, we tackled some of the most widely-known preprocessing steps such as scaling, label encoding, and missing value imputation. We finished with some machine learning to predict if a person's application for a credit card would get approved or not given some information about that person.

In [34]:
# Instantiate GridSearchCV with the required parameters

grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Use scaler to rescale X and assign it to rescaledX
rescaledX = scaler.fit_transform(X)

# Fit data to grid_model
grid_result = grid_model.fit(rescaledX, y)

# Summarize results
best_score, best_params = grid_result.best_score_, grid_result.best_params_
print("Best: %f using %s" % (best_score, best_params))



Best: 0.852174 using {'max_iter': 100, 'tol': 0.01}


References

https://developers.google.com/machine-learning/crash-course/classification/accuracy

https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html

https://www.dataschool.io/machine-learning-with-scikit-learn/
